In [ ]:
import pandas as pd
import psycopg2

from dotenv import load_dotenv
import os
load_dotenv()


WIKI_REPO_PATH = os.getenv('WIKI_REPO_PATH')

WIKI_DB_HOST = os.getenv('WIKI_DB_HOST')
WIKI_DB_NAME = os.getenv('WIKI_DB_NAME')
WIKI_DB_PORT = os.getenv('WIKI_DB_PORT')
WIKI_DB_USER = os.getenv('WIKI_DB_USER')
WIKI_DB_PASS = os.getenv('WIKI_DB_PASS')


# # Connect to PostgreSQL
# def open_conn():
#     '''Открытие подключения к БД в случае его разрыва'''
#     return psycopg2.connect(
#         dbname=WIKI_DB_NAME,
#         user=WIKI_DB_USER,
#         password=WIKI_DB_PASS,
#         host=WIKI_DB_HOST,
#         port=WIKI_DB_PORT
#     )

# Connect to PostgreSQL
def conn_pg():
    try:
        conn = psycopg2.connect(
            dbname=WIKI_DB_NAME,
            user=WIKI_DB_USER,
            password=WIKI_DB_PASS,
            host=WIKI_DB_HOST,
            port=WIKI_DB_PORT
        )
        print("Connected to PostgreSQL")
        cur = conn.cursor()
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        print("PostgreSQL version:", db_version)
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
        
conn_pg()



# # Open connection
# conn = open_conn()


Connected to PostgreSQL
PostgreSQL version: ('PostgreSQL 17.5 (Ubuntu 17.5-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [ ]:
conn = conn_pg()

cur = conn.cursor()

# fetch wiki pages paths
cur.execute("SELECT path FROM public.pages;")
wiki_page_paths = pd.DataFrame(cur.fetchall(), columns=['wiki_page_path'])

# fetch wiki assets paths
cur.execute('''
    WITH RECURSIVE folder_paths AS (
        SELECT 
            id,
            name dir_name,
            slug,
            "parentId",
            name::text AS full_path
        FROM "assetFolders"
        WHERE "parentId" IS NULL
        UNION ALL
        SELECT 
            af.id,
            af.name,
            af.slug,
            af."parentId",
            fp.full_path || '/' || af.name AS full_path
        FROM "assetFolders" af
        JOIN folder_paths fp ON af."parentId" = fp.id
    )
    select
        concat(fp.full_path, '/', a.filename) as path
    from folder_paths fp
    join public.assets a
    on a."folderId" = fp.id;
''')

wiki_asset_paths = pd.DataFrame(cur.fetchall(), columns=['wiki_asset_path'])



In [49]:
wiki_asset_paths

,wiki_asset_path
0,assets/silas/comm/0a77a6c15c85f60b517f06e05be7...
1,assets/silas/comm/0e432f756f9725ffaf0801faefd1...
2,assets/silas/comm/1000036249 1.jpg
3,assets/silas/comm/1000036249.jpg
4,assets/silas/comm/12121-3-350x350.jpg
...,...
805,.obsidian/app.json
806,assets/pub/0bd6dd80c5c6e8286e30051dfbd24753.jpg
807,assets/pub/narodnaya-smehovaya-kultura-v-filos...
808,assets/silas/comm/cf3d86a23f6202a0d53b4048f1db...


In [52]:
# find all files ending with .md in the wiki repo
import os
def get_repo_paths(repo_path: str, file_ext: str='.md') -> pd.DataFrame:
    """Find all pages and assets in the repository path and return their relative paths. 
    Pages paths will have the file extension removed."""
    
    repo_asset_paths = []
    repo_page_paths = []
    for root, dirs, files in os.walk(repo_path):
        for file_name in files:
            if root.endswith('assets'):
                # replace \ with /
                file_path = file_path.replace('\\', '/')
                # replace repo_path with empty string
                file_path = file_path.replace(repo_path, '')
                # remove leading slash
                file_path = file_path.lstrip('/')
                repo_asset_paths.append(file_path)
            else:
                if file_name.endswith(file_ext):
                    # remove extension
                    file_path = os.path.join(root, file_name)[:-len(file_ext)]
                    # replace \ with /
                    file_path = file_path.replace('\\', '/')
                    # replace repo_path with empty string
                    file_path = file_path.replace(repo_path, '')
                    # remove leading slash
                    file_path = file_path.lstrip('/')
                    repo_page_paths.append(file_path)
    
    repo_asset_paths = pd.DataFrame(repo_asset_paths, columns=['repo_asset_path'])
    repo_page_paths = pd.DataFrame(repo_page_paths, columns=['repo_page_path'])

    return repo_asset_paths, repo_page_paths

In [53]:
repo_asset_paths, repo_page_paths = get_repo_paths(WIKI_REPO_PATH)

In [55]:
def get_missing_page_paths(repo_paths: pd.DataFrame, wiki_paths: pd.DataFrame, source: str) -> pd.DataFrame:
    """Find paths that are present in one of the dataframes but not in both."""
    merged_df = pd.merge(repo_paths, wiki_paths, left_on=f'repo_{source}_path', right_on=f'wiki_{source}_path', suffixes=('_repo', '_wiki'), how='outer')
    
    # list of paths with missing for both repo and wiki
    missing_page_paths = merged_df[
        (merged_df[f'repo_{source}_path'].isnull()) | (merged_df[f'wiki_{source}_path'].isnull())
    ]
    
    return missing_page_paths

In [59]:
page_errors = get_missing_page_paths(repo_page_paths, wiki_page_paths, 'page')
asset_errors = get_missing_page_paths(repo_asset_paths, wiki_asset_paths, 'asset')

In [57]:
page_errors

,repo_page_path,wiki_page_path
6,Templates/note,NaN
7,assets/silas/c_2/Семь_тел_человека.excalidraw,NaN
16,Общее/Борьба_за_свободу_в эпоху_постмодерна,NaN
28,Общее/Как_быть_счастливым_в_разных культурах,NaN
97,Общее/Персоны/Аркадий_Гайдар,NaN
111,Общее/Персоны/Евгений_Смотрин,NaN
117,Общее/Персоны/Изабелла_Гуревич,NaN
119,Общее/Персоны/Иммануил_Великовский,NaN
141,Общее/Персоны/Сократ,NaN


In [60]:
asset_errors

,repo_asset_path,wiki_asset_path
0,NaN,.obsidian/app.json
1,NaN,.obsidian/appearance.json
2,NaN,.obsidian/community-plugins.json
3,NaN,.obsidian/core-plugins.json
4,NaN,assets/background.jpg
...,...,...
806,NaN,cache/ff22d3be46d27342760e9f9f63777cd444c7b847...
807,NaN,cache/ff73d07ced1b42d7b4e9f99569e0291023730028...
808,NaN,cache/ffeb16411a47b2912892d31b8b46662c574eda7c...
809,home,NaN
